In [1]:
import pandas as pd
import pickle
import time
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy import spatial
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import sys
import joblib
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split

In [3]:
trade_off_cluster = 1000
bert_len = 768

In [4]:
class EmbedBaseline:
    def __init__(self, model, tokenizer, trade_off_cluster,device="cpu"):
        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        self.model = self.model.to(self.device)
    def fit_cluster(self, X):
        self.kmeans.fit(X)
    def get_bert_embed(self, text):
        text = "[CLS] " + text + " [SEP]"
        tokenized_text = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        tokenized_text = tokenized_text.to(self.device)
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.model.device) for k, v in tokenized_text.items()})

        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy().astype(float)

    def calculate_cosine_similarity(self, vac_des, cv_vec):
        return float(spatial.distance.cosine(vac_des, cv_vec))
    
    def predict_one(self, df_res, location, vac_des):
        first = df_res[(df_res["City"] == location) | (df_res["microcat_name"] =="Удаленная работа") ]
        vac_des_embd = self.get_bert_embed(vac_des)      
        similarity = []
        for idx, elem in enumerate(first["embd"].values):
            similarity.append(float(self.calculate_cosine_similarity(vac_des_embd, elem)))
        return similarity
    def predict(self, df_res, df_vac):
        answer = []
        for index, row in df_vac.iterrows():
            answer.append(self.predict_one(df_res, row["City"], row["vac_des"]))
        return answer

In [2]:
# file = open("drive/My Drive/data/avito_cv2vac_with_ranks_clear.pq", "rb")
# unpickler = pickle.Unpickler(file)
# df = unpickler.load()
df = pd.read_parquet("avito_cv2vac_with_ranks_clear.pq")

In [5]:
df["vac_des"].iloc[3]

'! В гипермаркет требуются сборщики заказов.\n\n! Условия:\n- График 2/2 (график можно менять);\n- Режим работы с 9.00 до 21.00;\n- ВЫПЛАТЫ ЕЖЕНЕДЕЛЬНО!\n- Ставка за полную смену до 2000р.;\n- Оплата почасовая + за каждую позицию в заказе;\n- Комплексный обед;\n- Можно без опыта, стажировка до 4-х часов;\n- Наличие медкнижки или готовность ее оформить;\n- Телефон на базе Андроид или Айфон;\n- Хорошие условия, выплаты вовремя!\n\n! Обязанности: сборка онлайн-заказов на территории гипермаркета.\n\nПИШИТЕ ПОЖАЛУЙСТА! Оставляйте отклик. Я обязательно с вами свяжусь!'

In [7]:
def take_embd(df):
    """
        Эмбединги в df очень странной рамзерности (размер,)
        приведем к (размер, длин эмбединга)
    """
    data = df["embd"].values
    data_new = np.zeros((data.shape[0], bert_len))
    for i, elem in enumerate(data):
        data_new[i] = elem
    return data_new

In [8]:
def intersection_list(array1, array2): 
    return len(set(array1) & set(array2)) 


In [9]:
# подгружаем данные
with open("val_id.pickle", 'rb') as f:
    val_id = pickle.load(f)

with open("test_id.pickle", 'rb') as f:
    test_id = pickle.load(f)

with open("train_id.pickle", 'rb') as f:
    train_id = pickle.load(f)
# x, test_id = train_test_split(np.array([i for i in range(len(df))]),test_size=0.05,train_size=0.95)
# train_id, val_id = train_test_split(x,test_size = 0.15,train_size =0.85)
intersection_list(train_id, val_id), intersection_list(train_id, test_id), intersection_list(test_id, val_id), len(train_id), len(val_id), len(test_id)

(0, 0, 0, 75488, 3775, 15097)

In [10]:
rubert_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased", max_len=512)

rubert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
%%time
baseline_model = EmbedBaseline(model=rubert, tokenizer=rubert_tokenizer, trade_off_cluster=trade_off_cluster, device="cuda")
# baseline_model.fit_cluster(take_embd(df.iloc[train_id]))


Wall time: 420 ms


In [12]:
val = df.iloc[val_id]
test = df.iloc[test_id]
val

,User_id,Item_id,EventDate,last_resume_id,event_name,Region,City,session_hash,microcat_name,Platform_id,res_title,res_des,vac_title,vac_des,label,rank,max_rank,vac_id
index,,,,,,,,,,,,,,,,,,
6,1.839085e+11,1.121613e+12,2023-03-13 03:02:37,1.421296e+12,Чат / События полученные через AMQP / Отправка...,Татарстан,Набережные Челны,3.857504e+18,Вахтовый метод,1.0,Любая работа с ежедневной оплатой,"Работал подсобником, грузчиком, сборщиком на к...","Грузчик - комплектовщик (вахта 15/15, с прожив...","! ВАХТА в МОСКОВСКОЙ ОБЛАСТИ !\n\nВОЗМОЖНО, БЕ...",1,3,21,3
66065,3.666616e+08,1.346818e+12,2023-03-13 11:03:36,4.358598e+11,Пользовательские события / Объявления / Просмо...,Рязанская область,Рязань,7.729033e+18,Сменный график,3.0,Водитель персональный,Москвич ищет постоянную и стабильную работу пе...,Заправщик АЗС,МЫ ПРЕДЛАГАЕМ:\n- достойная заработная плата (...,1,1,1,28677
63,4.252633e+11,8.682000e+11,2023-03-13 11:01:52,8.592930e+11,Пользовательские события / Объявления / Просмо...,Московская область,Люберцы,2.141965e+18,Полный день,2.0,Шиномонтажник грузовой-легковой правщик вулкан,Ищу работу жилательно на постоянной основе име...,Шиномонтажник на сезон,!!! ВНИМАНИЕ !!! \n НА СЕЗОН\nНА ШИНОМОНТАЖ с ...,1,1,1,19
66108,3.770781e+08,1.427300e+12,2023-03-13 16:48:38,1.432071e+12,Чат / События полученные через AMQP / Отправка...,Москва,Москва,6.790265e+18,Полный день,1.0,"Виталий Непомнящий53 годаМосква, Жулебино +7 9...",Техник водоподготовки и эксплуатации бассейнов...,Прораб / Начальник участка,"МЫ СТРОИТЕЛЬНАЯ КОМПАНИЯ ""МАСТЕРА РЕМОНТА"". СП...",1,20,22,28688
75,9.430048e+11,1.431397e+12,2023-03-13 18:40:00,1.430377e+12,Чат / События полученные через AMQP / Отправка...,Ростовская область,Ростов-на-Дону,3.552222e+18,Неполный день,3.0,Любая,Большое желание работать\n*проблемы с деньгами*,Продавец консультант во взрослый магазин,"!!! Для отклика на вакансию напишите ваше имя,...",1,1,1,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65834,1.050981e+12,1.432071e+12,2023-03-13 14:23:15,1.398460e+12,Чат / События полученные через AMQP / Отправка...,Иркутская область,Иркутск,2.565956e+18,Неполный день,4.0,"Любая работа, подработка","Добрая, ответственность присутствует",Начинающий специалист,"МТС — это цифровые сервисы, которые делают жиз...",1,4,4,28570
65862,2.900648e+11,1.313510e+12,2023-03-13 15:11:24,6.407048e+11,Пользовательские события / Объявления / Просмо...,Москва,Москва,6.542361e+18,Полный день,4.0,Электромонтажник,"Работал в сфере обслуживания, Электромонтажник...",Мастер-универсал по ремонту квартир,"МЫ - Группа Компаний ""ФЕНИКС"", занимаемся внут...",1,1,1,28585
65880,9.427560e+11,1.432072e+12,2023-03-13 14:21:54,1.378080e+12,Чат / События полученные через AMQP / Отправка...,Санкт-Петербург,Санкт-Петербург,3.716423e+18,Неполный день,3.0,"Любая работа, подработка",Работник торгового зала,Актер/Актриса для фильмов,"МЫ В ПОИСКЕ АКТЕРОВ, ТАКЖЕ РАССМОТРИМ КАНДИДАТ...",1,10,19,28590


In [ ]:
%%time
# baseline_model.predict(df.iloc[test_id], test.iloc[0:1000])

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 10.5 µs


In [13]:
from tqdm import tqdm
from sklearn.metrics import ndcg_score

def count_ndcg_for_all_resumes(model, df, k=None):
  '''
  ранжируем к вакансии вообще все резюме из кластера 
  это походит на боевые условия, но имхо, ждать тут высокого ndcg не стоит из-за разреженнности векторов
  '''
  if not k:
    k = df.shape[0]

  df = df.drop_duplicates(['vac_des', 'res_des'])
  df.index = range(df.shape[0])
  if df.shape[0] > 0:
    ndcg_vals = []
    for vac in tqdm(df['vac_des'].unique()):
      possible_res = df[df['vac_des'] == vac][['res_des', 'rank']]
      if possible_res.shape[0] < 3:
        continue
      true_rank_vector = np.zeros((df.shape[0]))
      true_rank_vector[possible_res.index] = possible_res['rank']
      model_preds = model_inference_dataset(model, df)

      if len(true_rank_vector) < k:
        ndcg_vals.append(ndcg_score([true_rank_vector], [model_preds]))
      else:
        ndcg_vals.append(ndcg_score([true_rank_vector[:k]], [model_preds[:k]]))

    return ndcg_vals
  
def model_inference_one_vac(model, dataset, vac):
  '''
  Инференс на 1 вакансии. У вас функция будет отличаться, очевидно
  '''
  # проваливаемся в кластер по локации

  first = dataset[(dataset["City"] == vac["City"].values[0]) | (dataset["microcat_name"] =="Удаленная работа") ]
  vac_des_embd = model.get_bert_embed(vac["vac_des"].values[0])      
  similarity = []
  for idx, elem in enumerate(first["embd"].values):
      similarity.append(float(model.calculate_cosine_similarity(vac_des_embd, elem)))
  return similarity, np.arange(first.shape[0])

def model_inference_dataset(model, dataset):
  '''
  Инференс на всех вакансиях в dataset
  '''
  # for_aply = dataset["res_des"]
  # dataset["embd"] = for_aply.apply(lambda x: model.get_bert_embed(x))
  preds = np.zeros((dataset.shape[0]))
  for vac in dataset['vac_des'].unique():
    model_pred, pred_ind = model_inference_one_vac(model, dataset, dataset[dataset["vac_des"] == vac].iloc[0:1])
    preds[pred_ind] = model_pred

  return preds

In [17]:
val["embd"] = val["res_des"].apply(lambda x: baseline_model.get_bert_embed(x))

In [18]:
# count_ndcg_for_all_resumes(baseline_model, val)

In [ ]:
print(np.array([0.14725709949600815,
 0.17376877667712315,
 0.1936783373767222,
 0.17111546963656915,
 0.19909888540428594,
 0.15592774492996175,
 0.22836361097144708,
 0.18205998870816956,
 0.15213522601956936,
 0.16100625006287156,
 0.12382298949786935,
 0.10811417737922908,
 0.12747343740582082,
 0.13227203145688501,
 0.1582337886346083,
 0.11641869797973554,
 0.10910471309061946,
 0.10872441367648832,
 0.14383240226896693,
 0.15363497754896177,
 0.11541896441727761,
 0.1333216031942639,
 0.11408736064839083,
 0.13748925376803908,
 0.13421929363886637,
 0.10521509754427878,
 0.1264861717812378,
 0.10167296706752417,
 0.1321693124754539,
 0.09805443538460248,
 0.11990006064137111,
 0.11146200026262347,
 0.1347127517191224,
 0.13014458231447465,
 0.1278789778160138,
 0.15788635455519748,
 0.1332276967743591,
 0.14005978835501517,
 0.1717854793711504,
 0.13872264815629212,
 0.11841193804509696,
 0.11518972845337816,
 0.14056094795577634,
 0.11164610277371147,
 0.17853772918255045,
 0.1497258185533368,
 0.13070216744243737,
 0.13009745813393359,
 0.11889455194454882,
 0.14101892374921962,
 0.11712155226689284,
 0.10862545681776735,
 0.182359860090119,
 0.12122069983319937,
 0.11053996660018273,
 0.11433200392720363,
 0.1119785978737291,
 0.10589769117849591,
 0.11958344289866951,
 0.11738530496283041,
 0.13667287817233448,
 0.09963395534730798]).mean())

0.13532410635984177


In [22]:
def count_ndcg_for_appropriate_resumes(model, df, k=None):
  '''
  ранжируем только подходящие к вакансии резюме
  то есть делаем предикт только для них и смотрим, как алгоритм умеет сортировать "хорошие" сэмплы
  '''
  if not k:
    k = float('inf')

  df = df.drop_duplicates(['vac_des', 'res_des'])
  df.index = range(df.shape[0])
  if df.shape[0] > 0:
    ndcg_vals = []
    for vac in tqdm(df['vac_des'].unique()):
      possible_res = df[df['vac_des'] == vac][['City', 'microcat_name', 'vac_des', 'res_des', 'rank', "embd"]]
      if possible_res.shape[0] < 3:
        continue

      true_rank_vector = possible_res['rank'].values
      model_probs = model_inference_dataset(model, possible_res)

      if len(true_rank_vector) < k:
        ndcg_vals.append(ndcg_score([true_rank_vector], [model_probs]))
      else:
        ndcg_vals.append(ndcg_score([true_rank_vector[:k]], [model_probs[:k]]))

    return ndcg_vals

In [23]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3775 entries, 6 to 65962
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   User_id         3775 non-null   float64       
 1   Item_id         3775 non-null   float64       
 2   EventDate       3775 non-null   datetime64[ns]
 3   last_resume_id  3775 non-null   float64       
 4   event_name      3775 non-null   object        
 5   Region          3775 non-null   object        
 6   City            3775 non-null   object        
 7   session_hash    3775 non-null   float64       
 8   microcat_name   3775 non-null   object        
 9   Platform_id     3775 non-null   float64       
 10  res_title       3775 non-null   object        
 11  res_des         3775 non-null   object        
 12  vac_title       3775 non-null   object        
 13  vac_des         3775 non-null   object        
 14  label           3775 non-null   int64         
 15  ran

In [25]:
mod = count_ndcg_for_appropriate_resumes(baseline_model, val, k=10)

100%|█████████████████████████████████████████████████████████████████████████████| 3334/3334 [00:03<00:00, 998.21it/s]


In [ ]:
print(np.array([0.9578591031587825,
 0.84521003836195,
 0.8367554619279153,
 0.991261454613035,
 0.6814295933652503,
 0.8521914221634458,
 0.664356599293337,
 0.7927791295647445,
 0.6857884052562886,
 0.6238326542353767,
 0.9401510514007158,
 0.7578712286761339,
 0.8135263458948311,
 0.97299309943651,
 0.6439343450497917,
 0.961064189869163,
 0.9477696816191689,
 0.7142276794041407,
 0.8304821219177927,
 0.8762792332516216,
 0.8368535027687898,
 0.744098377747129,
 0.9422677283143694,
 0.8594719041203874,
 0.8892747572163654,
 0.6995457069242146,
 0.9256694036042509,
 0.6803315233476523,
 0.9046520362567405,
 1.0,
 0.9948044697928312,
 1.0,
 0.7035149594498942,
 0.8747452126554871,
 0.8467627054653891,
 0.6634018075447283,
 0.8748247413816372,
 0.7752638165375105,
 0.7482384150445662,
 0.9517909894306406,
 0.8409576715178898,
 0.929210688905503,
 0.7567898430874557,
 0.7713228241772939,
 0.954879024320319,
 0.7241153539729269,
 0.7646072162067911,
 0.8314151456138487,
 0.6883953414846187,
 0.752881359533751,
 0.8206328812304725,
 0.7489260514202083,
 0.7448491574799937,
 0.677344870107489,
 0.7180234846285897,
 0.7065037775013757,
 0.7611679368476773,
 0.7217377867016326,
 0.9491002413480794,
 0.8253883893925729,
 0.914141385893304,
 0.5854622128883687]).mean())

0.8144052829084311


In [ ]:
count_ndcg_for_all_resumes(baseline_model, val, k = 10)

100%|██████████| 3334/3334 [1:36:39<00:00,  1.74s/it]


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [27]:
np.array(mod).mean()

0.8152958991605201